In [1]:
!pip install --upgrade d2l

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.9/58.9 kB 1.1 MB/s eta 0:00:00
  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.7/111.7 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.3/98.3 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.7/124.7 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.5/93.5 kB 1.9 MB/s eta 0:00:00
Using cached jedi-0.19.1-py2.py3-none-any.whl (1.6 MB)
  Attempting uninstall: requests
    Found existing installation: requests 2.32.3
    Uninsta

# Ponderada - Tradução Automática


In [2]:
import os
import tensorflow as tf
from d2l import tensorflow as d2l

**Obtendo os dados**

In [3]:
d2l.DATA_HUB['fra-eng'] = (d2l.DATA_URL + 'fra-eng.zip',
                           '94646ad1522d915e7b0f9296181140edcf86a4f5')

def read_data_nmt():
  """Load the English-French dataset."""
  data_dir = d2l.download_extract('fra-eng')
  with open(os.path.join(data_dir, 'fra.txt'), 'r') as f:
    return f.read()

raw_text = read_data_nmt()
print(raw_text[:75])

Go.	Va !
Hi.	Salut !
Run!	Cours !
Run!	Courez !
Who?	Qui ?
Wow!	Ça alors !



**Processando os dados**

In [4]:
def preprocess_nmt(text):
  """Preprocess the English-French dataset."""
  def no_space(char, prev_char):
      return char in set(',.!?') and prev_char != ' '

  # Replace non-breaking space with space, and convert uppercase letters to
  # lowercase ones
  text = text.replace('\u202f', ' ').replace('\xa0', ' ').lower()
  # Insert space between words and punctuation marks
  out = [' ' + char if i > 0 and no_space(char, text[i - 1]) else char
          for i, char in enumerate(text)]
  return ''.join(out)

text = preprocess_nmt(raw_text)
print(text[:80])

go .	va !
hi .	salut !
run !	cours !
run !	courez !
who ?	qui ?
wow !	ça alors !


**Tokenização**

Esta seção define a função **tokenize_nmt**, que é responsável por dividir os dados de texto em tokens individuais (palavras ou subpalavras). 

A tokenização é um passo importante na preparação dos dados de texto para o treinamento de um modelo.

In [5]:
def tokenize_nmt(text, num_examples=None):
  """Tokenize the English-French dataset."""
  source, target = [], []
  for i, line in enumerate(text.split('\n')):
    if num_examples and i > num_examples:
      break
    parts = line.split('\t')
    if len(parts) == 2:
      source.append(parts[0].split(' '))
      target.append(parts[1].split(' '))

  return source, target

source, target = tokenize_nmt(text)
source[:6], target[:6]

([['go', '.'],
  ['hi', '.'],
  ['run', '!'],
  ['run', '!'],
  ['who', '?'],
  ['wow', '!']],
 [['va', '!'],
  ['salut', '!'],
  ['cours', '!'],
  ['courez', '!'],
  ['qui', '?'],
  ['ça', 'alors', '!']])

## Vocabulário

Esta seção constrói o vocabulário a partir do texto tokenizado. O vocabulário é um mapeamento de todos os tokens únicos encontrados no conjunto de dados para índices numéricos, que são usados durante o treinamento do modelo. 

Os parâmetros especificam a frequência mínima (min_freq=2), o que significa que apenas palavras que aparecem pelo menos duas vezes são incluídas, e reservam tokens especiais (<pad>, <bos>, <eos>) para preenchimento, início e fim de uma sequência.

In [9]:
src_vocab = d2l.Vocab(source, min_freq=2,
                      reserved_tokens=['<pad>', '<bos>', '<eos>'])
len(src_vocab)

10012

**Carregando o Conjunto de Dados**

In [10]:
def truncate_pad(line, num_steps, padding_token):
  """Truncate or pad sequences."""
  if len(line) > num_steps:
    return line[:num_steps]  # Truncate

  return line + [padding_token] * (num_steps - len(line))  # Pad

truncate_pad(src_vocab[source[0]], 10, src_vocab['<pad>'])

[3919, 80, 208, 208, 208, 208, 208, 208, 208, 208]

In [12]:
def build_array_nmt(lines, vocab, num_steps):
  """Transform text sequences of machine translation into minibatches."""
  lines = [vocab[l] for l in lines]
  lines = [l + [vocab['<eos>']] for l in lines]

  array = tf.constant([truncate_pad(
    l, num_steps, vocab['<pad>']) for l in lines])

  valid_len = tf.reduce_sum(
    tf.cast(array != vocab['<pad>'], tf.int32), 1)

  return array, valid_len

## Unificando todas as coisas

Esta função integra todos os passos anteriores, incluindo carregamento de dados, tokenização e criação do vocabulário, para preparar o conjunto de dados para o treinamento em lotes. 

In [13]:
def load_data_nmt(batch_size, num_steps, num_examples=600):
  """Return the iterator and the vocabularies of the translation dataset."""
  text = preprocess_nmt(read_data_nmt())
  source, target = tokenize_nmt(text, num_examples)

  src_vocab = d2l.Vocab(source, min_freq=2,
                        reserved_tokens=['<pad>', '<bos>', '<eos>'])

  tgt_vocab = d2l.Vocab(target, min_freq=2,
                        reserved_tokens=['<pad>', '<bos>', '<eos>'])

  src_array, src_valid_len = build_array_nmt(source, src_vocab, num_steps)
  tgt_array, tgt_valid_len = build_array_nmt(target, tgt_vocab, num_steps)

  data_arrays = (src_array, src_valid_len, tgt_array, tgt_valid_len)
  data_iter = d2l.load_array(data_arrays, batch_size)

  return data_iter, src_vocab, tgt_vocab


In [14]:
train_iter, src_vocab, tgt_vocab = load_data_nmt(batch_size=2, num_steps=8)

for X, X_valid_len, Y, Y_valid_len in train_iter:
  print('X:', tf.cast(X, tf.int32))
  print('valid lengths for X:', X_valid_len)

  print('Y:', tf.cast(Y, tf.int32))
  print('valid lengths for Y:', Y_valid_len)

  break

X: tf.Tensor(
[[ 25 173  59   7   4   5   5   5]
 [ 60  90   0   4   5   5   5   5]], shape=(2, 8), dtype=int32)
valid lengths for X: tf.Tensor([5 4], shape=(2,), dtype=int32)
Y: tf.Tensor(
[[143 124  68  13   7   4   5   5]
 [ 28   6   0   4   5   5   5   5]], shape=(2, 8), dtype=int32)
valid lengths for Y: tf.Tensor([6 4], shape=(2,), dtype=int32)
